<a href="https://colab.research.google.com/github/EdmilsonSantana/llm-vehicle-repair/blob/main/Assistente_do_Mecanico_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install docx ipywidgets

In [2]:
from docx import Document
from docx.text.paragraph import Paragraph
from docx.shared import Pt
import os
import re
import modal
import json
import pandas as pd
import numpy as np
from helper import load_json

## Processando apostilas da Universidade do Automóvel

In [3]:
RAW_DIR = './raw_data'
BASE_DIR = './data'

In [4]:
files = os.listdir(RAW_DIR)

In [4]:
print(f'{len(files)} documents')

4 documents


In [5]:
files

['1 - Motor de Comb. Interna - Otto.docx',
 '2 - Sistema de Transmissão.docx',
 '3 - Metrologia Mec. Automotiva.docx',
 '4 - Suspensão, Direção e Freios.docx']

In [42]:
document_content_list = {filename: get_document_content(
    filename) for filename in files}

In [43]:
def show_document_sections(document_content_list):
    for filename, document_content in document_content_list.items():
        print(f'{filename} = ', list(document_content.keys()))

In [44]:
show_document_sections(document_content_list)

1 - Motor de Comb. Interna - Otto.docx =  ['TITLE', 'CICLO OTTO', 'Conteúdos', 'INTRODUÇÃO', 'MOTOR DE COMBUSTÃO', 'MOTOR DE COMBUSTÃO EXTERNA', 'MOTOR DE COMBUSTÃO INTERNA', 'MOTORES DE DOIS TEMPOS', 'MOTORES DE QUATRO TEMPOS', 'CABEÇOTE DO MOTOR', 'TIPOS DE CABEÇOTE', 'DISTRIBUIÇÃO MECÂNICA', 'DIAGRAMA DE VÁLVULAS', 'POSIÇÃO DO COMANDO E TIPO DE MOTOR', 'ESTRUTURA DA DISTRIBUIÇÃO MECÂNICA', 'ÁRVORE DE COMANDO DE VÁLVULAS', 'TUCHOS', 'CONJUNTO DE BALANCINS', 'MANUTENÇÃO', 'VÁLVULAS', 'CONSTITUIÇÃO DA VÁLVULA', 'DISPOSITIVOS DE MONTAGEM DAS VÁLVULAS', 'PEÇAS MÓVEIS', 'BIELA', 'CASQUILHO', 'ÊMBOLO', 'ANÉIS DE SEGMENTO', 'ÁRVORE DE MANIVELAS', 'BLOCO DE MOTOR', 'TIPOS DE BLOCO', 'VOLANTE DO MOTOR', 'FUNÇÕES DO VOLANTE DO MOTOR', 'DIMENSÕES E DESEMPENHO DO MOTOR', 'LUBRIFICAÇÃO', 'ATRITO', 'LUBRIFICANTES', 'SISTEMA DE LUBRIFICAÇÃO', 'SISTEMA DE LUBRIFICAÇÃO MISTO', 'SISTEMA DE CÁRTER SECO', 'CÁRTER', 'FILTRO DE ÓLEO', 'BOMBA DE ÓLEO', 'VÁLVULA REGULADORA DE PRESSÃO', 'BIBLIOGRAFIA']
2 - S

In [45]:
invalid_sections = [
    'TITLE',
    'CICLO OTTO',
    'Conteúdos', 
    'INTRODUÇÃO',
    'BIBLIOGRAFIA',
    'MECÂNICA DE VEÍCULOS LEVES',
    'SUMÁRIO',
    'INTRODUÇÃO',
    'METROLOGIA PARA MECÂNICA AUTOMOTIVA',
    'APRESENTAÇÃO',
    'REFERÊNCIAS',
    'REFERÊNCIAS BIBLIOGRÁFICAS'
]

In [46]:
for filename, document_content in document_content_list.items():
    for section in list(document_content.keys()):
        if section in invalid_sections or len(document_content[section]) == 0:
            document_content.pop(section)

In [47]:
show_document_sections(document_content_list)

1 - Motor de Comb. Interna - Otto.docx =  ['MOTOR DE COMBUSTÃO', 'MOTOR DE COMBUSTÃO EXTERNA', 'MOTOR DE COMBUSTÃO INTERNA', 'MOTORES DE DOIS TEMPOS', 'MOTORES DE QUATRO TEMPOS', 'CABEÇOTE DO MOTOR', 'TIPOS DE CABEÇOTE', 'DISTRIBUIÇÃO MECÂNICA', 'DIAGRAMA DE VÁLVULAS', 'POSIÇÃO DO COMANDO E TIPO DE MOTOR', 'ESTRUTURA DA DISTRIBUIÇÃO MECÂNICA', 'ÁRVORE DE COMANDO DE VÁLVULAS', 'TUCHOS', 'CONJUNTO DE BALANCINS', 'MANUTENÇÃO', 'VÁLVULAS', 'CONSTITUIÇÃO DA VÁLVULA', 'DISPOSITIVOS DE MONTAGEM DAS VÁLVULAS', 'PEÇAS MÓVEIS', 'BIELA', 'CASQUILHO', 'ÊMBOLO', 'ANÉIS DE SEGMENTO', 'ÁRVORE DE MANIVELAS', 'BLOCO DE MOTOR', 'TIPOS DE BLOCO', 'VOLANTE DO MOTOR', 'FUNÇÕES DO VOLANTE DO MOTOR', 'DIMENSÕES E DESEMPENHO DO MOTOR', 'LUBRIFICAÇÃO', 'ATRITO', 'LUBRIFICANTES', 'SISTEMA DE LUBRIFICAÇÃO', 'SISTEMA DE LUBRIFICAÇÃO MISTO', 'SISTEMA DE CÁRTER SECO', 'CÁRTER', 'FILTRO DE ÓLEO', 'BOMBA DE ÓLEO', 'VÁLVULA REGULADORA DE PRESSÃO']
2 - Sistema de Transmissão.docx =  ['TRANSMISSÃO ARTICULADA', 'SISTEMA 

In [48]:
with open(f'{BASE_DIR}/raw_documents.json', 'w', encoding='utf-8') as fp:
    json.dump(document_content_list, fp, ensure_ascii=False)

In [49]:
documents = []
id = 1
for filename, document_content in document_content_list.items():
    for section, sentences in document_content.items():
        context = " ".join(sentences)
        documents.append(dict(id=id, filename=filename, section=section, context=context))
        id += 1

In [50]:
with open(f'{BASE_DIR}/documents.json', 'w', encoding='utf-8') as fp:
    json.dump(documents, fp, ensure_ascii=False)

## Extraindo instruções a partir do conteúdo

In [5]:
! modal deploy model.py

- Creating objects...
| Creating objects...
/ Creating objects...
└── - Creating mount e:\Workspace\tcc-2022-2\model.py: Uploaded 0/0 inspected 
    files
\ Creating objects...
└── \ Creating mount e:\Workspace\tcc-2022-2\model.py: Building mount
/ Creating objects...
└── / Creating mount e:\Workspace\tcc-2022-2\model.py: Building mount
\ Creating objects...
└── 🔨 Created mount e:\Workspace\tcc-2022-2\model.py
/ Creating objects...
└── 🔨 Created mount e:\Workspace\tcc-2022-2\model.py
\ Creating objects...
└── 🔨 Created mount e:\Workspace\tcc-2022-2\model.py
/ Creating objects...
└── 🔨 Created mount e:\Workspace\tcc-2022-2\model.py
\ Creating objects...
└── 🔨 Created mount e:\Workspace\tcc-2022-2\model.py
/ Creating objects...
├── 🔨 Created mount e:\Workspace\tcc-2022-2\model.py
└── - Creating download_model_to_folder...
\ Creating objects...
├── 🔨 Created mount e:\Workspace\tcc-2022-2\model.py
└── | Creating download_model_to_folder...
/ Creating objects...
├── 🔨 Created mount e:\Works

In [6]:
inference = modal.Function.lookup('llm-inference', 'Model.generate')

### Extraindo tópicos dos documentos

In [7]:
documents = json.load(open(f'{BASE_DIR}/documents.json', 'r', encoding='utf-8'))

In [8]:
documents[0]

{'id': 1,
 'filename': '1 - Motor de Comb. Interna - Otto.docx',
 'section': 'MOTOR DE COMBUSTÃO',
 'context': 'Há muitos séculos, o homem vem se defrontando com problemas de transporte; nos primórdios da civilização foi utilizada a tração animal, porém muito dos esforços humanos foram dirigidos à construção de veículos que não dependessem da tração animal. Um dos primeiros resultados foi a invenção da máquina a vapor, conhecida também como motor de combustão externa, logo em seguida a sua invenção, foi instalada nos veículos já existentes (carruagens, triciclos e bicicletas). Consta que o primeiro desses veículos, um triciclo a vapor, foi construído na França, já em 1771. Esse triciclo iniciou um processo que não se interrompeu mais; a produção de veículos que têm propulsão própria. A máquina a vapor, entretanto, apresenta muitos problemas e inconvenientes: Seu baixo rendimento obriga-a a utilizar grandes quantidades de combustível e água; A produção de vapor em caldeiras, pelo calor 

In [9]:
n_topics = 5

topics_system_prompt = f"""
Given a document your goal is to extract {n_topics} topics that summarizes the content present in the document.
Here are some criterias that you have to follow to achieve your goal:

- The topics should be extracted in the same language as the document.
- You should not extract topics not related to vehicle repair and maintenance.
- You should not extract short topics.
- The output should be a JSON list of strings in the following format: ["topic", "topic", "topic"].

examples:

Document: O cuidado com o meio ambiente é uma preocupação global imprescindível para o desenvolvimento sustentável. No mundo dos automóveis, essa discussão é cada vez maior, com o aumento do rigor na legislação em diversos países - o que acelera a evolução de veículos de propulsão alternativa, como híbridos e elétricos. O aperto excessivo da porca afeta diretamente a vida útil do rolamento. Em rolamentos dianteiros selados, Airton, da Schaeffler, adverte para nunca substituir a graxa ou completar os espaços internos, porque pode haver uma reação entre as graxas com composições químicas diferentes, além de um aquecimento elevado no interior do rolamento.
Answer: [\"A importância global do cuidado com o meio ambiente\", \"A evolução dos veículos de propulsão alternativa\", \"O aumento da legislação ambiental nos países\", \"Impacto do aperto excessivo da porca na vida útil do rolamento\", \"Cuidados específicos com a manutenção de rolamentos dianteiros selados\"]
Document: Entre os vários parâmetros levados em conta no projeto da suspensão de um carro, os três principais ângulos medidos no alinhamento de veículos leves são convergência e divergência (paralelismo horizontal entre as rodas), câmber (ângulo de inclinação lateral da roda em relação ao eixo vertical) e o cáster (inclinação do eixo vertical referente ao centro da circunferência da roda). Mas antes de instalar o equipamento de medição, o primeiro procedimento que o mecânico deve fazer é analisar os sintomas.
Answer: [\"Parâmetros essenciais no projeto da suspensão de um carro\", \"Importância da análise dos sintomas antes da medição\", \"Convergência e divergência: impacto no alinhamento das rodas\", \"Câmber: influência do ângulo de inclinação lateral da roda\", \"Cáster: considerações sobre a inclinação do eixo vertical\"]"""

topics_user_prompt = """
Document: {document}
Answer:"""

In [89]:
documents

[{'id': 1,
  'filename': '1 - Motor de Comb. Interna - Otto.docx',
  'section': 'MOTOR DE COMBUSTÃO',
  'context': 'Há muitos séculos, o homem vem se defrontando com problemas de transporte; nos primórdios da civilização foi utilizada a tração animal, porém muito dos esforços humanos foram dirigidos à construção de veículos que não dependessem da tração animal. Um dos primeiros resultados foi a invenção da máquina a vapor, conhecida também como motor de combustão externa, logo em seguida a sua invenção, foi instalada nos veículos já existentes (carruagens, triciclos e bicicletas). Consta que o primeiro desses veículos, um triciclo a vapor, foi construído na França, já em 1771. Esse triciclo iniciou um processo que não se interrompeu mais; a produção de veículos que têm propulsão própria. A máquina a vapor, entretanto, apresenta muitos problemas e inconvenientes: Seu baixo rendimento obriga-a a utilizar grandes quantidades de combustível e água; A produção de vapor em caldeiras, pelo ca

In [90]:
def create_topics(documents):
    topics_list = []
    prompts = []
    failed_responses = []
    topic_id = 1
    for document in documents:
        document_id = document['id']
        document = document['context']
        prompts.append(topics_user_prompt.format(document=document))

    print(f"Inference with {len(prompts)} prompts")
    response = inference.remote(topics_system_prompt, prompts)
    print(f"Found response with {len(response)} items")

    for document, response_data in zip(documents, response):
        try:
            print(f"Parsing document {document['id']} with {len(document)} length")

            for topic in load_json(response_data):
                data = dict(id=topic_id,
                            document_id=document['id'],
                            topic=topic)
                print(f'{data=}')
                topics_list.append(data)
                topic_id += 1
        except Exception as error:
            print(f"{document_id=} {error=}")
            failed_responses.append((document_id, response_data, error))

    return topics_list, failed_responses

In [91]:
topics_list, failed_responses = create_topics(documents)

Inference with 133 prompts
Found response with 133 items
Parsing document 1 with 4 length
data={'id': 1, 'document_id': 1, 'topic': 'Problema de transporte no mundo'}
data={'id': 2, 'document_id': 1, 'topic': 'Inventiva da máquina a vapor'}
data={'id': 3, 'document_id': 1, 'topic': 'Carros e bicicletas a vapor'}
data={'id': 4, 'document_id': 1, 'topic': 'Inventiva da máquina endotérmica'}
data={'id': 5, 'document_id': 1, 'topic': 'Motor de combustão interna do ciclo Otto'}
data={'id': 6, 'document_id': 1, 'topic': 'Carl Benz e o primeiro automóvel'}
data={'id': 7, 'document_id': 1, 'topic': 'Como funciona o motor de combustão interna'}
Parsing document 2 with 4 length
data={'id': 8, 'document_id': 2, 'topic': 'Locomotiva a vapor'}
data={'id': 9, 'document_id': 2, 'topic': 'Funcionamento da locomotiva a vapor'}
data={'id': 10, 'document_id': 2, 'topic': 'Motor de combustão externa'}
data={'id': 11, 'document_id': 2, 'topic': 'Principais exemplos de triciclos a vapor'}
data={'id': 12, 'd

In [93]:
len(failed_responses)

0

In [100]:
with open(f'{BASE_DIR}/topics.json', 'w', encoding='utf-8') as fp:
    json.dump(topics_list, fp, ensure_ascii=False)

#### Validando tópicos extraidos

In [203]:
topics_validation_system_prompt = """
Given a context and a topic, classify how related the topic is to the context with a score between 0 and 1.0 and a reason for the score, considering 1.0 as a very related topic and 0 as an unrelated topic. Your output should be a JSON for an object that represents the score and the reason. It should have the fields "score" and "reason" where "score" is a float and "reason" is a string.

examples:

Topic: A importância global do cuidado com o meio ambiente
Context: O cuidado com o meio ambiente é uma preocupação global imprescindível para o desenvolvimento sustentável. No mundo dos automóveis, essa discussão é cada vez maior, com o aumento do rigor na legislação em diversos países - o que acelera a evolução de veículos de propulsão alternativa, como híbridos e elétricos. O aperto excessivo da porca afeta diretamente a vida útil do rolamento. Em rolamentos dianteiros selados, Airton, da Schaeffler, adverte para nunca substituir a graxa ou completar os espaços internos, porque pode haver uma reação entre as graxas com composições químicas diferentes, além de um aquecimento elevado no interior do rolamento.
Answer: {"score": 1.0, "reason": "O tópico do cuidado ambiental é abordado diretamente no contexto. A discussão sobre o desenvolvimento de veículos de propulsão alternativa na indústria automotiva é citada como uma consequência das preocupações ambientais globais. Portanto, o tópico está altamente relacionado ao contexto."}
Topic: Recomendações do grupo Schaeffler
Context: O cuidado com o meio ambiente é uma preocupação global imprescindível para o desenvolvimento sustentável. No mundo dos automóveis, essa discussão é cada vez maior, com o aumento do rigor na legislação em diversos países - o que acelera a evolução de veículos de propulsão alternativa, como híbridos e elétricos. O aperto excessivo da porca afeta diretamente a vida útil do rolamento. Em rolamentos dianteiros selados, Airton, da Schaeffler, adverte para nunca substituir a graxa ou completar os espaços internos, porque pode haver uma reação entre as graxas com composições químicas diferentes, além de um aquecimento elevado no interior do rolamento.
Answer: {"score": 0.1, "reason": "Embora o texto mencione Airton da Schaeffler, o foco principal está nas preocupações ambientais e seu impacto no desenvolvimento automotivo. As recomendações do grupo Schaeffler não são o tema central, resultando em uma pontuação baixa de relevância."}
Topic: Funções e tipos de válvulas de motor
Context: O cuidado com o meio ambiente é uma preocupação global imprescindível para o desenvolvimento sustentável. No mundo dos automóveis, essa discussão é cada vez maior, com o aumento do rigor na legislação em diversos países - o que acelera a evolução de veículos de propulsão alternativa, como híbridos e elétricos. O aperto excessivo da porca afeta diretamente a vida útil do rolamento. Em rolamentos dianteiros selados, Airton, da Schaeffler, adverte para nunca substituir a graxa ou completar os espaços internos, porque pode haver uma reação entre as graxas com composições químicas diferentes, além de um aquecimento elevado no interior do rolamento.
Answer: {"score": 0, "reason": "O tópico das funções e tipos de válvulas de motor não é mencionado ou sugerido no contexto fornecido. Portanto, está sem relação, resultando em uma pontuação de 0."}"""

topics_validation_user_prompt = """
Topic: {topic}
Context: {context}
Answer:"""

In [221]:
documents = json.load(open(f'{BASE_DIR}/documents.json', 'r', encoding='utf-8'))
topics_list = json.load(open(f'{BASE_DIR}/topics.json', 'r', encoding='utf-8'))

In [222]:

def validate_topics(documents, topics_list, splits=2):
    documents_df = pd.DataFrame(documents)
    topics_df = pd.DataFrame(topics_list)
    df_context_topic = pd.merge(
        documents_df, topics_df, left_on='id', right_on='document_id')
    test_scenarios = df_context_topic[['context', 'topic']].to_dict('records')
    response_list = []
    failed_responses = []

    for chunk in np.array_split(test_scenarios, splits):
        prompts = []
        for scenario in chunk:
            prompts.append(topics_validation_user_prompt.format(
                context=scenario['context'], topic=scenario['topic']))

        print(f"Inference with {len(prompts)} prompts")
        response = inference.remote(topics_validation_system_prompt, prompts)
        print(f"Found response with {len(response)} items")
        response_list.extend(response)

    for topic_data, response_data in zip(topics_list, response_list):
        try:
            id = topic_data['id']
            print(
                f"Validating topic {id} = '{topic_data['topic']}'")
            topic_data['validation'] = load_json(response_data)
        except Exception as error:
            print(f"{error=} {id=}")
            topic_data['validation'] = {}
            failed_responses.append((error, topic_data, response_data))
    return failed_responses

In [223]:
failed_responses = validate_topics(documents, topics_list, splits=10)

Inference with 67 prompts
Found response with 67 items
Inference with 67 prompts
Found response with 67 items
Inference with 67 prompts
Found response with 67 items
Inference with 67 prompts
Found response with 67 items
Inference with 67 prompts
Found response with 67 items
Inference with 67 prompts
Found response with 67 items
Inference with 67 prompts
Found response with 67 items
Inference with 67 prompts
Found response with 67 items
Inference with 66 prompts
Found response with 66 items
Inference with 66 prompts
Found response with 66 items
Validating topic 1 = 'Problema de transporte no mundo'
Validating topic 2 = 'Inventiva da máquina a vapor'
Validating topic 3 = 'Carros e bicicletas a vapor'
Validating topic 4 = 'Inventiva da máquina endotérmica'
Validating topic 5 = 'Motor de combustão interna do ciclo Otto'
Validating topic 6 = 'Carl Benz e o primeiro automóvel'
Validating topic 7 = 'Como funciona o motor de combustão interna'
Validating topic 8 = 'Locomotiva a vapor'
Validati

In [226]:
len(failed_responses)

5

In [227]:
for item in failed_responses:
    print(item[2])

{"score": 1.0, "reason": "A descrição fornecida é sobre o funcionamento de um motor de 4 tempos, que abrange todos os passos do ciclo de trabalho, incluindo admissão, compressão, combustão e escapamento, bem como o papel dos cilindros e volante no sistema. Isso corresponde à questão de "funcionamento de um motor de 4 tempos". Assim, o tópico tem uma alta relevância ao contexto."}
{"score": 1.0, "reason": "The topic of "Embolos" is directly addressed in the context as one of the components of a motor that are inspected during maintenance. The context provides information about the function and wear of the embolos, making the topic very relevant."}
{"score": 1.0, "reason": "The topic of "manutenção da válvula reguladora de pressão" is closely related to the context, as it discusses the maintenance of an important component of the engine's lubrication system. The context provides information about the purpose and function of the pressure regulator valve, and mentions the need for periodic

In [ ]:
with open(f'{BASE_DIR}/topics.json', 'w', encoding='utf-8') as fp:
    json.dump(topics_list, fp, ensure_ascii=False)

In [34]:
documents = json.load(open(f'{BASE_DIR}/documents.json', 'r', encoding='utf-8'))
topics_list = json.load(open(f'{BASE_DIR}/topics.json', 'r', encoding='utf-8'))

In [35]:
threshold = 0.5
failed_test_scenarios = []
for topic_data in topics_list:
    if isinstance(topic_data['validation'], float):
        score = topic_data['validation']
    elif 'score' not in topic_data['validation']:
        continue
    else:
       score = topic_data['validation'].get('score')
    
    if score < threshold:
        failed_test_scenarios.append(topic_data)

In [36]:
len(failed_test_scenarios)

36

In [38]:
failed_test_scenarios
documents_df = pd.DataFrame(documents)
failed_test_scenarios_df = pd.DataFrame(failed_test_scenarios)
df_context_failed_scenarios = pd.merge(documents_df, failed_test_scenarios_df, left_on='id', right_on='document_id')

In [39]:
df_context_failed_scenarios.drop(columns=['id_x', 'id_y', 'document_id', 'filename'], inplace=True)

In [40]:
with open(f'{BASE_DIR}/failed_test_scenarios.json', 'w', encoding='utf-8') as fp:
    json.dump( df_context_failed_scenarios.to_dict('records'), fp, ensure_ascii=False)

#### Extraindo questões e respostas dos tópicos e documentos

In [11]:
documents_df = pd.DataFrame(documents)
topics_df = pd.DataFrame(topics_list)
df_context_topic = pd.merge(documents_df, topics_df, left_on='id', right_on='document_id')
df_context_topic.drop(columns=['id_x'], inplace=True)
df_context_topic.rename(columns={'id_y': 'id'}, inplace=True)

In [12]:
questions_prompt = """
You are a specialist in vehicle repair and maintenance and your goal is to extract questions and answers related to a certain topic from a document.
Here are some criterias that you have to follow to achieve your goal:

- You should not create questions for contents not related to vehicle repair and maintenance.
- You should include the topic in the question so that it is possible to understand what the question is about.
- The answers should only contain the content originating from the document.
- You should create as many questions as possible, up to a maximum of 5 questions.
- The language used to generate questions and answers should always be Brazilian Portuguese.
- Your response should always be only a well-formed JSON array with the questions and answers.

examples:

Topic: Rolamentos dianteiros
Document: O cuidado com o meio ambiente é uma preocupação global imprescindível
para o desenvolvimento sustentável. No mundo dos automóveis, essa discussão é cada vez maior,
com o aumento do rigor na legislação em diversos países - o que acelera a evolução de veículos de propulsão alternativa,
como híbridos e elétricos. O aperto excessivo da porca afeta diretamente a vida útil do rolamento.
Em rolamentos dianteiros selados, Airton, da Schaeffler, adverte para nunca
substituir a graxa ou completar os espaços internos, porque pode haver uma reação
entre as graxas com composições químicas diferentes, além de um aquecimento elevado no interior do rolamento.

Answer:
[
    {{
        "question": "Posso apertar a porca do rolamento ?",
        "answer": "O aperto excessivo da porca afeta diretamente a vida útil do rolamento."
    }},
    {{
        "question": "Devo substituir a graxa de rolamentos dianteiros selados ?",
        "answer": "Em rolamentos dianteiros selados, nunca substitua a graxa ou complete os espaços internos, porque pode haver uma reação entre as graxas com composições químicas diferentes, além de um aquecimento elevado no interior do rolamento."
    }}
]

Topic: {topic}
Document: {document}
Answer:"""

In [13]:
topics_list = df_context_topic.to_dict('records')
topics_list[0]

{'filename': '1 - Motor de Comb. Interna - Otto.docx',
 'section': 'MOTOR DE COMBUSTÃO',
 'context': 'Há muitos séculos, o homem vem se defrontando com problemas de transporte; nos primórdios da civilização foi utilizada a tração animal, porém muito dos esforços humanos foram dirigidos à construção de veículos que não dependessem da tração animal. Um dos primeiros resultados foi a invenção da máquina a vapor, conhecida também como motor de combustão externa, logo em seguida a sua invenção, foi instalada nos veículos já existentes (carruagens, triciclos e bicicletas). Consta que o primeiro desses veículos, um triciclo a vapor, foi construído na França, já em 1771. Esse triciclo iniciou um processo que não se interrompeu mais; a produção de veículos que têm propulsão própria. A máquina a vapor, entretanto, apresenta muitos problemas e inconvenientes: Seu baixo rendimento obriga-a a utilizar grandes quantidades de combustível e água; A produção de vapor em caldeiras, pelo calor gerado nas

In [14]:
def create_queries(topics_list, splits=2):
    queries_list = []
    query_id = 1
    failed_responses = []
    response_list = []

    for chunk in np.array_split(topics_list, splits):
        prompts = []
        for topic_data in chunk:
            topic = topic_data['topic']
            document = topic_data['context']
            prompts.append(questions_prompt.format(topic=topic, document=document))

        print(f"Inference with {len(prompts)} prompts")
        response = inference.remote(prompts)
        print(f"Found response with {len(response)} items")
        response_list.extend(response)

    for topic_data, response_data in zip(topics_list, response_list):
        try:
            print(
                f"Extracting question related to '{topic_data['topic']}'")

            for query in json.loads(response_data):
                data = dict(id=query_id,
                            document_id=topic_data['document_id'],
                            topic_id=topic_data['id'],
                            question=query['question'],
                            answer=query['answer'])
                print(f'{data=}')
                queries_list.append(data)
                query_id += 1
        except Exception as error:
            print(f"{error=} {topic_data=} {response_data=} ")
            failed_responses.append((topic_data, response_data))

    return queries_list, failed_responses

In [15]:
len(topics_list)

645

In [16]:
queries_list, failed_responses = create_queries(topics_list, 10)

Inference with 65 prompts
Found response with 65 items
Inference with 65 prompts
Found response with 65 items
Inference with 65 prompts
Found response with 65 items
Inference with 65 prompts
Found response with 65 items
Inference with 65 prompts
Found response with 65 items
Inference with 64 prompts
Found response with 64 items
Inference with 64 prompts
Found response with 64 items
Inference with 64 prompts
Found response with 64 items
Inference with 64 prompts
Found response with 64 items
Inference with 64 prompts
Found response with 64 items
Extracting question related to 'O evolução dos veículos de transporte'
error=JSONDecodeError('Extra data: line 23 column 1 (char 1196)') topic_data={'filename': '1 - Motor de Comb. Interna - Otto.docx', 'section': 'MOTOR DE COMBUSTÃO', 'context': 'Há muitos séculos, o homem vem se defrontando com problemas de transporte; nos primórdios da civilização foi utilizada a tração animal, porém muito dos esforços humanos foram dirigidos à construção de v

In [17]:
len(failed_responses)

32

In [19]:
len(queries_list)

2224

In [20]:
with open(f'{BASE_DIR}/questions.json', 'w', encoding='utf-8') as fp:
    json.dump(queries_list, fp, ensure_ascii=False)